# Лабораторная работа №1
### Конечномерная оптимизация, градиентные методы

In [10]:
from typing import Tuple, Mapping

import numpy as np
from matplotlib import pyplot as plt
import sklearn as skl

#GDF
def my_GD(f: Mapping, df: Mapping, x0: np.ndarray, lr: float = 0.01, T: int = 100) -> Tuple [np.ndarray, np.float32]:
    '''Моя простейшая реализация градиентного спуска.
    
    Args:
        f (Mapping): Функционал для оптимизации
        df (Mapping): Градиент оптимизирующего функционала
        x0 (np.ndarray): Стартовая точка
        lr (float): Скорость обучения. Default=0,01.
        T (int): Количество итераций.
    
    Returns:
        Tuple [np.ndarray, np.float32]: (x_optimal, f(x_optimal)).
    
    '''
    for i in range(100):
        x_new = x_old - lr*df(x_old)
        
    return x_new, f(x_new)
    

In [23]:
#функция
def Himmelblau(x: np.float32, y: np.float32) -> np.float64:
    '''
    Функция Химмельблау
    
    Args:
        x(np.ndarray): Аргумент 1
        y(np.ndarray): Аргумент 2
        
    Returns:
        np.float64: Результат функции
    '''
    return np.square(np.square(x)+y-11)+np.square(x+np.square(y)-7);

#градиент
def Himmelblau_Grad(x: np.ndarray, y: np.ndarray) -> np.float64:
    return np.array((4*x*(np.square(x)+y-11)+2*(x+np.square(y)-7)),(2*(np.square(x)+y-11)+4*y*(x+np.square(y)-7)));

Himmelblau(2.5, 3.4), Himmelblau_Grad(2.5, 3.4)

(51.66609999999998, array(0.62))